<a href="https://colab.research.google.com/github/vincenzoaltavilla/thesis_project/blob/main/thesis_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DOWNLOAD DATASET DA KAGGLE**

LINK: https://www.kaggle.com/code/chrisferentinos/premier-league-championship-team-analysis/notebook

In [1]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'player-scores:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1071543%2F7910805%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240520%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240520T104920Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D07fcb2d1e41c6bd75feeccff6e1b8c9beec882f59cb91784327ef563e6305415e410dab20d9bd591dde31c94b9a011a620b1630694a5a7da8fadd19f910d605308ee4e13cf4d352987e081ffb39fe48479139c3a0e3ba6e0427c3d9e1f773ce2e2d68ccc1f56c8e3100863d2f11b1cf505a50bab037ffb2993a8b6f258f28a587049a31bf8f778af5433e47e340ed38402a7fef0b46f2ec2478205a1aff950e5ed48f48e63c55ce66c29ef20eee3d83a92bdd72201802f3016164f02c196eeb941ed02aa1d194c720170a434bb92fb796c169f4d664bdf66f0330cd815cc902b78798aee7ed106b243689393504f281daea4db56a888410399d590e6a3eea831'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

[==================================================] 152669203 bytes downloaded
Downloaded and uncompressed: player-scores


# **IMPORT LIBRERIE, ACQUISIZIONE DATASET E AGGIUSTAMENTI PRELIMINARI SUI TIPI DI DATI**



In [281]:
import os
import warnings
warnings.filterwarnings('ignore')

import plotly.io as pio
pio.renderers.default='colab'

import pandas as pd
pd.set_option('display.max.columns', None)

def european_format(x):
    return '{:,.2f}'.format(x).replace(',', 'X').replace('.', ',').replace('X', '.')

pd.set_option('display.float_format', european_format)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime

appearances = pd.read_csv("/kaggle/input/player-scores/appearances.csv")                #used
club_games = pd.read_csv("/kaggle/input/player-scores/club_games.csv")                  #used
clubs = pd.read_csv("/kaggle/input/player-scores/clubs.csv")                            #used
competitions = pd.read_csv("/kaggle/input/player-scores/competitions.csv")              #used
game_events = pd.read_csv("/kaggle/input/player-scores/game_events.csv")                #used
#game_lineups = pd.read_csv("/kaggle/input/player-scores/game_lineups.csv")
#games = pd.read_csv("/kaggle/input/player-scores/games.csv")
player_valuations = pd.read_csv("/kaggle/input/player-scores/player_valuations.csv")    #used
players = pd.read_csv("/kaggle/input/player-scores/players.csv")                        #used

players['date_of_birth'] = pd.to_datetime(players['date_of_birth'])
appearances['date'] = pd.to_datetime(appearances['date'])
game_events['date'] = pd.to_datetime(game_events['date'])
player_valuations['date'] = pd.to_datetime(player_valuations['date'])

# **INIZIO LAVORO**

## **Data cleaning & feature engineering**

### **Aggiustamento tabella "Players"**

In [282]:
mean_height = players['height_in_cm'].mean()
players['height_in_cm'] = players['height_in_cm'].fillna(mean_height)
players['height_in_cm'] = players['height_in_cm'].astype(int)
players['foot'] = players['foot'].fillna('both')

players = players.drop(columns=['url','image_url','agent_name','last_season','current_club_id',
                      'player_code','country_of_birth','city_of_birth', 'current_club_name',
                      'current_club_domestic_competition_id', 'market_value_in_eur',
                      'highest_market_value_in_eur', 'contract_expiration_date',
                      'first_name', 'last_name'])

#altezza media per valori nulli, ambidestro per piedi nulli

### **Join fra "Players", "Appearances", "Clubs" e "Competitions"**

In [283]:
players_appearances = pd.merge(players, appearances, on='player_id', how='inner')
players_appearances = players_appearances.drop(columns=['player_current_club_id','appearance_id',
                                                        'name', 'competition_id'])

In [284]:
players_appearances = pd.merge(players_appearances, clubs, left_on='player_club_id', right_on='club_id', how='inner')
players_appearances = players_appearances.drop(columns=['player_club_id','club_code','squad_size',
                                                        'average_age', 'foreigners_number',
                                                        'foreigners_percentage',
                                                        'national_team_players', 'stadium_name',
                                                        'stadium_seats', 'net_transfer_record',
                                                        'last_season', 'filename',
                                                        'url', 'coach_name', 'total_market_value'])
players_appearances = players_appearances.rename(columns={"name": "club_name"})

In [285]:
players_appearances = pd.merge(players_appearances, competitions, left_on='domestic_competition_id', right_on='competition_id', how='left')
players_appearances = players_appearances.drop(columns=['domestic_competition_id','competition_code',
                                                        'name', 'sub_type', 'type', 'country_id',
                                                        'competition_id', 'domestic_league_code',
                                                        'confederation', 'url'])

In [286]:
players_appearances = players_appearances.rename(columns={"country_name": "club_national_league",
                                                          "is_major_national_league": "is_top5_league"})
players_appearances = players_appearances[['player_id', 'player_name', 'position', 'sub_position',
                                          'foot', 'height_in_cm', 'country_of_citizenship',
                                          'date_of_birth', 'club_id', 'club_name',
                                          'club_national_league', 'is_top5_league', 'game_id', 'date',
                                          'yellow_cards', 'red_cards', 'goals', 'assists',
                                          'minutes_played']]

### **Different type of goals**

In [287]:
goal_events = game_events[game_events['type']== 'Goals']
goal_events = goal_events.drop(columns=['game_event_id', 'player_in_id', 'type', 'player_assist_id',
                                        'minute'])
mask = goal_events['description'].str.contains('Own-goal', case=False)
goal_events = goal_events[~mask]

In [288]:
kind_of_goals = ['Header', 'Right-footed', 'Left-footed', 'Penalty', 'Long distance kick', 'Free kick']

for kind_of_goal in kind_of_goals:
    mask = goal_events['description'].str.contains(kind_of_goal, case=False)
    goal_events.loc[mask, 'description'] = kind_of_goal

import re
regex_other_goals = '|'.join(kind_of_goals)

mask = ~goal_events['description'].str.contains(regex_other_goals, case=False)
goal_events.loc[mask, 'description'] = 'Other'

In [289]:
goal_events['right_footed_goals'] = 0
goal_events['left_footed_goals'] = 0
goal_events['long_distance_goals'] = 0
goal_events['direct_free_kick_goals'] = 0
goal_events['penalty_goals'] = 0
goal_events['header_goals'] = 0
goal_events['other_kind_of_goals'] = 0

goal_events.loc[goal_events['description'] == 'Header', 'header_goals'] = 1
goal_events.loc[goal_events['description'] == 'Right-footed', 'right_footed_goals'] = 1
goal_events.loc[goal_events['description'] == 'Left-footed', 'left_footed_goals'] = 1
goal_events.loc[goal_events['description'] == 'Penalty', 'penalty_goals'] = 1
goal_events.loc[goal_events['description'] == 'Long distance kick', 'long_distance_goals'] = 1
goal_events.loc[goal_events['description'] == 'Free kick', 'direct_free_kick_goals'] = 1
goal_events.loc[goal_events['description'] == 'Other', 'other_kind_of_goals'] = 1

#.sort_values(by=['date'], ascending=False)

In [290]:
goal_events = goal_events.groupby(['date', 'game_id', 'player_id']).agg({
    'club_id': 'first',
    'description': 'first',
    'right_footed_goals': 'sum',
    'left_footed_goals': 'sum',
    'long_distance_goals': 'sum',
    'direct_free_kick_goals': 'sum',
    'penalty_goals': 'sum',
    'header_goals': 'sum',
    'other_kind_of_goals': 'sum',
}).reset_index()

goal_events.rename(columns=dict(zip(goal_events.columns[:5], ['goal_'+str(col) for col in goal_events.columns[:5]])), inplace=True)

### **Player valuations**

In [291]:
player_valuations = player_valuations.drop(columns=['current_club_id','player_club_domestic_competition_id'])
player_valuations['year'] = player_valuations ['date'].dt.year

player_valuations = player_valuations.groupby(['player_id', 'year']).agg({
    'date': 'first',
    'market_value_in_eur': 'mean'
}).reset_index()

player_valuations = player_valuations.drop(columns=['date'])
player_valuations.rename(columns={"market_value_in_eur": "avg_yearly_market_value_in_eur"}, inplace=True)

## **First important join: "Players_appearances" & "Goal_events"**

In [292]:
players_appearances_detailed = pd.merge(players_appearances, goal_events, left_on=['game_id', 'player_id'], right_on=['goal_game_id', 'goal_player_id'], how='left')
players_appearances_detailed = players_appearances_detailed.drop(columns=['goal_date', 'goal_game_id',
                                                                          'goal_player_id', 'goal_club_id'])

In [293]:
players_appearances_detailed.loc[players_appearances_detailed['goals'] == 0, ['goal_description',
                                                                              'right_footed_goals',
                                                                              'left_footed_goals',
                                                                              'long_distance_goals',
                                                                              'direct_free_kick_goals',
                                                                              'penalty_goals',
                                                                              'header_goals',
                                                                              'other_kind_of_goals']] = 0

players_appearances_detailed = players_appearances_detailed.drop(columns=['goal_description'])

players_appearances_detailed['right_footed_goals'] = players_appearances_detailed['right_footed_goals'].fillna(0).astype(int)
players_appearances_detailed['left_footed_goals'] = players_appearances_detailed['left_footed_goals'].fillna(0).astype(int)
players_appearances_detailed['long_distance_goals'] = players_appearances_detailed['long_distance_goals'].fillna(0).astype(int)
players_appearances_detailed['direct_free_kick_goals'] = players_appearances_detailed['direct_free_kick_goals'].fillna(0).astype(int)
players_appearances_detailed['penalty_goals'] = players_appearances_detailed['penalty_goals'].fillna(0).astype(int)
players_appearances_detailed['header_goals'] = players_appearances_detailed['header_goals'].fillna(0).astype(int)
players_appearances_detailed['other_kind_of_goals'] = players_appearances_detailed['other_kind_of_goals'].fillna(0).astype(int)

In [294]:
first_stats = players_appearances_detailed[['player_id', 'player_name', 'position', 'sub_position',
                                            'foot',	'height_in_cm',	'country_of_citizenship',
                                            'date_of_birth', 'club_id', 'club_name',
                                            'club_national_league', 'is_top5_league', 'date',
                                            'game_id', 'minutes_played', 'goals', 'assists',
                                            'right_footed_goals', 'left_footed_goals',
                                            'header_goals', 'direct_free_kick_goals',
                                            'penalty_goals', 'long_distance_goals',
                                            'other_kind_of_goals', 'yellow_cards',
                                            'red_cards',]]

## **Second important join: "First_stats" & "Club_games"**

In [295]:
intermediate_stats = pd.merge(first_stats, club_games, on=['game_id', 'club_id'], how='inner')
intermediate_stats = intermediate_stats.drop(columns=['own_goals', 'own_position', 'own_manager_name',
                                                      'opponent_id', 'opponent_position',
                                                      'opponent_manager_name', 'hosting'])

In [296]:
intermediate_stats['is_clean_sheet'] = 1
intermediate_stats.loc[intermediate_stats['opponent_goals'] > 0, 'is_clean_sheet'] = 0
intermediate_stats['year'] = intermediate_stats['date'].dt.year
intermediate_stats['age'] = intermediate_stats['year'] - intermediate_stats['date_of_birth'].dt.year
intermediate_stats['age'] = intermediate_stats['age'].fillna(0).astype(int)

In [297]:
intermediate_stats = intermediate_stats.sort_values(by=['date'], ascending=[True])

## **First group by & add calculated features**

In [298]:
grouped_stats = intermediate_stats.groupby(['player_id', 'year']).agg({
    'player_name': 'first',
    'position': 'first',
    'sub_position': 'first',
    'foot': 'first',
    'height_in_cm': 'first',
    'country_of_citizenship': 'first',
    'date_of_birth': 'first',
    'club_id': 'first',
    'club_name': 'first',
    'club_national_league': 'first',
    'is_top5_league': 'first',
    'date': 'first',
    'game_id': 'count',
    'minutes_played': 'sum',
    'goals': 'sum',
    'assists': 'sum',
    'right_footed_goals': 'sum',
    'left_footed_goals': 'sum',
    'header_goals': 'sum',
    'direct_free_kick_goals': 'sum',
    'penalty_goals': 'sum',
    'long_distance_goals': 'sum',
    'other_kind_of_goals': 'sum',
    'yellow_cards': 'sum',
    'red_cards': 'sum',
    'opponent_goals': 'sum',
    'is_win': 'sum',
    'is_clean_sheet': 'sum',
    'age': 'first'
}).reset_index()


In [299]:
stats = grouped_stats[['player_id', 'year', 'player_name', 'position', 'sub_position',
                       'foot', 'height_in_cm',	'country_of_citizenship',
                       'date_of_birth', 'club_id', 'club_name',
                       'club_national_league', 'is_top5_league',
                       'age', 'game_id', 'minutes_played', 'is_win', 'goals', 'assists',
                       'right_footed_goals', 'left_footed_goals',
                       'header_goals', 'direct_free_kick_goals',
                       'penalty_goals', 'long_distance_goals',
                       'other_kind_of_goals', 'opponent_goals', 'is_clean_sheet', 'yellow_cards',
                       'red_cards']]

stats.rename(columns={"game_id": "games_played", "is_win": "games_won", "is_clean_sheet": "n_of_clean_sheets"}, inplace=True)
stats.loc[stats['position'] != "Goalkeeper", 'n_of_clean_sheets'] = 0

stats['goals_scored_per_match'] = ((stats['goals'] / stats['games_played'])).round(2)
stats['percentage_of_win'] = ((stats['games_won'] / stats['games_played']) * 100).round(2)
stats['percentage_of_clean_sheets'] = ((stats['n_of_clean_sheets'] / stats['games_played']) * 100).round(2)
stats['conceded_goals_per_match'] = ((stats['opponent_goals'] / stats['games_played'])).round(2)

stats = stats.dropna()

## **Third important join: "Stats" & "Player_valuations"**

In [300]:
stats_with_values = pd.merge(stats, player_valuations, on=['player_id', 'year'], how='inner')

# **Print**

In [303]:
stats_with_values
#[stats_with_values['player_name'] == 'Cristiano Ronaldo']
#.sort_values(by=['goals'], ascending=False).head(50)
#stats_with_values[stats_with_values.isnull().any(axis=1)]

,player_id,year,player_name,position,sub_position,foot,height_in_cm,country_of_citizenship,date_of_birth,club_id,club_name,club_national_league,is_top5_league,age,games_played,minutes_played,games_won,goals,assists,right_footed_goals,left_footed_goals,header_goals,direct_free_kick_goals,penalty_goals,long_distance_goals,other_kind_of_goals,opponent_goals,n_of_clean_sheets,yellow_cards,red_cards,goals_scored_per_match,percentage_of_win,percentage_of_clean_sheets,conceded_goals_per_match,avg_yearly_market_value_in_eur
0,10,2012,Miroslav Klose,Attack,Centre-Forward,right,184,Germany,1978-06-09,398,Società Sportiva Lazio S.p.A.,Italy,True,34,20,1483,13,11,1,7,1,2,0,0,0,1,14,0,6,0,"0,55","65,00","0,00","0,70","6.000.000,00"
1,10,2013,Miroslav Klose,Attack,Centre-Forward,right,184,Germany,1978-06-09,398,Società Sportiva Lazio S.p.A.,Italy,True,35,29,2052,12,9,4,4,1,2,0,0,0,2,36,0,3,0,"0,31","41,38","0,00","1,24","3.000.000,00"
2,10,2014,Miroslav Klose,Attack,Centre-Forward,right,184,Germany,1978-06-09,398,Società Sportiva Lazio S.p.A.,Italy,True,36,31,1766,15,8,6,3,1,0,0,0,0,4,40,0,3,0,"0,26","48,39","0,00","1,29","1.000.000,00"
3,10,2015,Miroslav Klose,Attack,Centre-Forward,right,184,Germany,1978-06-09,398,Società Sportiva Lazio S.p.A.,Italy,True,37,36,2429,18,12,8,6,0,4,0,0,0,2,38,0,6,0,"0,33","50,00","0,00","1,06","1.000.000,00"
4,10,2016,Miroslav Klose,Attack,Centre-Forward,right,184,Germany,1978-06-09,398,Società Sportiva Lazio S.p.A.,Italy,True,38,20,1078,8,8,6,6,0,1,0,1,0,0,25,0,1,0,"0,40","40,00","0,00","1,25","1.000.000,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85671,1186012,2023,Bakary Haidara,Defender,Centre-Back,right,182,Cote d'Ivoire,2005-06-01,968,Koninklijke Voetbal Club Westerlo,Belgium,False,18,5,352,2,0,0,0,0,0,0,0,0,0,9,0,3,0,"0,00","40,00","0,00","1,80","225.000,00"
85672,1187799,2023,Franciszek Franczak,Midfield,Right Midfield,right,173,Poland,2007-08-16,2578,Saint Johnstone Football Club,Scotland,False,16,4,233,1,0,0,0,0,0,0,0,0,0,6,0,1,0,"0,00","25,00","0,00","1,50","75.000,00"
85673,1187799,2024,Franciszek Franczak,Midfield,Right Midfield,right,173,Poland,2007-08-16,2578,Saint Johnstone Football Club,Scotland,False,17,3,149,1,0,0,0,0,0,0,0,0,0,2,0,0,0,"0,00","33,33","0,00","0,67","150.000,00"
85674,1196380,2024,Berhan Deniz,Attack,Centre-Forward,right,182,Türkiye,2004-11-25,152,Samsunspor,Turkey,False,20,1,3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,"0,00","0,00","0,00","1,00","75.000,00"


In [280]:
stats_with_values.to_csv('stats_with_values.csv', index=False)